# Chi Shift Comparison: Constant vs Physics Model

This notebook compares the traditional constant chi shift model with the new physics-based calculation. We'll demonstrate the differences in simulation accuracy and readout fidelity predictions.

## Background

Traditional dispersive readout simulators use simplified constant chi shifts like `[0, -0.25, -0.5, -0.75]` MHz. The new physics model calculates these shifts based on actual transmon parameters, providing more accurate modeling for:

- State discrimination optimization
- Readout fidelity predictions
- Protocol development and calibration

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from leeq.theory.simulation.numpy.dispersive_readout.simulator import (
    DispersiveReadoutSimulatorSyntheticData
)
from leeq.theory.simulation.numpy.dispersive_readout.physics import ChiShiftCalculator
from sklearn.mixture import GaussianMixture
import warnings
warnings.filterwarnings('ignore')  # Suppress sklearn warnings for cleaner output

# Set up matplotlib
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## 1. Setup Simulators with Different Chi Models

We'll create two simulators: one with constant chi shifts and one with physics-based calculation.

In [ ]:
# Common parameters
f_r = 7000              # Resonator frequency (MHz)
f_q = 5000              # Qubit frequency (MHz)
kappa = 1.0             # Resonator linewidth (MHz)
anharmonicity = -250    # Transmon anharmonicity (MHz)
coupling_strength = 100 # Coupling strength (MHz)

# Readout pulse parameters
pulse_params = {
    'width': 2.0,           # 2 μs readout
    'sampling_rate': 1000,  # 1 GS/s
    'amp': 1.0,
    'noise_std': 0.05       # 5% noise level
}

# Create simulator with constant chi shifts (legacy)
sim_constant = DispersiveReadoutSimulatorSyntheticData(
    f_r=f_r,
    kappa=kappa,
    chis=[0, -0.25, -0.5, -0.75],  # Traditional constant values
    use_physics_model=False,
    **pulse_params
)

# Create simulator with physics-based chi shifts
sim_physics = DispersiveReadoutSimulatorSyntheticData.from_physics_model(
    f_r=f_r,
    f_q=f_q,
    anharmonicity=anharmonicity,
    coupling_strength=coupling_strength,
    kappa=kappa,
    **pulse_params
)

print("Simulator Comparison:")
print("\nConstant Model Chi Shifts (MHz):")
for i, chi in enumerate(sim_constant.chis):
    print(f"  χ_{i} = {chi:.3f}")

print("\nPhysics Model Chi Shifts (MHz):")
for i, chi in enumerate(sim_physics.chis):
    print(f"  χ_{i} = {chi:.3f}")

print("\nDifferences (Physics - Constant):")
for i in range(len(sim_constant.chis)):
    diff = sim_physics.chis[i] - sim_constant.chis[i]
    print(f"  Δχ_{i} = {diff:.3f} MHz ({diff/abs(sim_constant.chis[i])*100 if sim_constant.chis[i] != 0 else 0:.1f}% change)")

## 2. Signal Trace Comparison

Let's compare the readout signals generated by both models for different qubit states.

In [ ]:
# Generate readout traces for different states
f_probe = f_r  # Probe at resonator frequency
num_traces = 100  # Number of traces per state for statistics
states_to_test = [0, 1, 2, 3]

# Store final I,Q values for each state and model
iq_constant = {state: [] for state in states_to_test}
iq_physics = {state: [] for state in states_to_test}

# Generate multiple traces for statistics
for state in states_to_test:
    for _ in range(num_traces):
        # Constant model
        trace_const = sim_constant._simulate_trace(state, f_probe, pulse_params['noise_std'])
        final_iq_const = np.sum(trace_const)  # Integrated signal
        iq_constant[state].append(final_iq_const)
        
        # Physics model
        trace_phys = sim_physics._simulate_trace(state, f_probe, pulse_params['noise_std'])
        final_iq_phys = np.sum(trace_phys)  # Integrated signal
        iq_physics[state].append(final_iq_phys)

# Plot I-Q distributions
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

colors = ['blue', 'red', 'green', 'purple']
markers = ['o', 's', '^', 'D']

# Constant model I-Q plot
for i, state in enumerate(states_to_test):
    iq_vals = np.array(iq_constant[state])
    I_vals = np.real(iq_vals)
    Q_vals = np.imag(iq_vals)
    ax1.scatter(I_vals, Q_vals, c=colors[i], marker=markers[i], 
               alpha=0.6, s=30, label=f'|{state}⟩')

ax1.set_xlabel('In-phase (I)')
ax1.set_ylabel('Quadrature (Q)')
ax1.set_title('Constant Chi Model')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.axis('equal')

# Physics model I-Q plot
for i, state in enumerate(states_to_test):
    iq_vals = np.array(iq_physics[state])
    I_vals = np.real(iq_vals)
    Q_vals = np.imag(iq_vals)
    ax2.scatter(I_vals, Q_vals, c=colors[i], marker=markers[i], 
               alpha=0.6, s=30, label=f'|{state}⟩')

ax2.set_xlabel('In-phase (I)')
ax2.set_ylabel('Quadrature (Q)')
ax2.set_title('Physics-Based Chi Model')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.axis('equal')

plt.tight_layout()
plt.show()

# Calculate state separation distances
def calculate_separation(iq_data_1, iq_data_2):
    """Calculate mean separation between two state distributions."""
    mean_1 = np.mean(iq_data_1)
    mean_2 = np.mean(iq_data_2)
    return abs(mean_1 - mean_2)

print("State Separation Analysis:")
print("\nConstant Model Separations:")
for i in range(len(states_to_test)-1):
    sep = calculate_separation(iq_constant[i], iq_constant[i+1])
    print(f"  |{i}⟩ - |{i+1}⟩: {sep:.3f}")

print("\nPhysics Model Separations:")
for i in range(len(states_to_test)-1):
    sep = calculate_separation(iq_physics[i], iq_physics[i+1])
    print(f"  |{i}⟩ - |{i+1}⟩: {sep:.3f}")

## 3. Readout Fidelity Comparison

We'll use Gaussian mixture models to quantify the readout fidelity for both chi models.

In [ ]:
def calculate_readout_fidelity(iq_data_dict, states=[0, 1]):
    """
    Calculate readout fidelity using Gaussian mixture model.
    
    Args:
        iq_data_dict: Dictionary with state -> list of I+jQ values
        states: States to include in fidelity calculation
        
    Returns:
        fidelity: Average state discrimination fidelity
        confusion_matrix: Classification confusion matrix
    """
    # Prepare training data
    X_train = []
    y_train = []
    
    for state in states:
        iq_vals = np.array(iq_data_dict[state])
        # Use real and imaginary parts as features
        features = np.column_stack([np.real(iq_vals), np.imag(iq_vals)])
        X_train.extend(features)
        y_train.extend([state] * len(features))
    
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    
    # Fit Gaussian mixture model
    gmm = GaussianMixture(n_components=len(states), random_state=42)
    gmm.fit(X_train)
    
    # Predict states
    y_pred = gmm.predict(X_train)
    
    # Calculate confusion matrix
    confusion = np.zeros((len(states), len(states)))
    for true_state, pred_state in zip(y_train, y_pred):
        true_idx = states.index(true_state)
        pred_idx = pred_state  # GMM returns 0, 1, 2, ...
        if pred_idx < len(states):
            confusion[true_idx, pred_idx] += 1
    
    # Normalize confusion matrix
    confusion_norm = confusion / confusion.sum(axis=1, keepdims=True)
    
    # Calculate average fidelity (diagonal elements)
    fidelity = np.mean(np.diag(confusion_norm))
    
    return fidelity, confusion_norm

# Calculate fidelities for different state pairs
state_pairs = [[0, 1], [1, 2], [2, 3], [0, 1, 2], [0, 1, 2, 3]]

print("Readout Fidelity Comparison:")
print("\nState Pair\t\tConstant Model\tPhysics Model\tImprovement")
print("-" * 65)

for states in state_pairs:
    # Constant model fidelity
    fid_const, conf_const = calculate_readout_fidelity(iq_constant, states)
    
    # Physics model fidelity
    fid_phys, conf_phys = calculate_readout_fidelity(iq_physics, states)
    
    improvement = (fid_phys - fid_const) * 100  # Percentage point improvement
    
    states_str = "-".join([f"|{s}⟩" for s in states])
    print(f"{states_str:<15}\t{fid_const:.3f}\t\t{fid_phys:.3f}\t\t{improvement:+.2f}%")

# Detailed analysis for |0⟩ vs |1⟩ discrimination
fid_01_const, conf_01_const = calculate_readout_fidelity(iq_constant, [0, 1])
fid_01_phys, conf_01_phys = calculate_readout_fidelity(iq_physics, [0, 1])

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 10))

# Confusion matrices
im1 = ax1.imshow(conf_01_const, cmap='Blues', vmin=0, vmax=1)
ax1.set_title(f'Constant Model\nFidelity: {fid_01_const:.3f}')
ax1.set_xlabel('Predicted State')
ax1.set_ylabel('True State')
for i in range(2):
    for j in range(2):
        ax1.text(j, i, f'{conf_01_const[i,j]:.3f}', ha='center', va='center', 
                color='white' if conf_01_const[i,j] > 0.5 else 'black', fontweight='bold')

im2 = ax2.imshow(conf_01_phys, cmap='Blues', vmin=0, vmax=1)
ax2.set_title(f'Physics Model\nFidelity: {fid_01_phys:.3f}')
ax2.set_xlabel('Predicted State')
ax2.set_ylabel('True State')
for i in range(2):
    for j in range(2):
        ax2.text(j, i, f'{conf_01_phys[i,j]:.3f}', ha='center', va='center',
                color='white' if conf_01_phys[i,j] > 0.5 else 'black', fontweight='bold')

# State distributions with decision boundary
for ax, iq_data, model_name in [(ax3, iq_constant, 'Constant'), (ax4, iq_physics, 'Physics')]:
    # Plot state distributions
    for state, color in [(0, 'blue'), (1, 'red')]:
        iq_vals = np.array(iq_data[state])
        I_vals = np.real(iq_vals)
        Q_vals = np.imag(iq_vals)
        ax.scatter(I_vals, Q_vals, c=color, alpha=0.6, s=20, label=f'|{state}⟩')
    
    ax.set_xlabel('In-phase (I)')
    ax.set_ylabel('Quadrature (Q)')
    ax.set_title(f'{model_name} Model |0⟩ vs |1⟩')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.axis('equal')

plt.tight_layout()
plt.show()

## 4. Parameter Sensitivity Analysis

Let's see how the differences between models change with various system parameters.

In [ ]:
def compare_models_vs_parameter(param_name, param_values, base_params):
    """
    Compare constant vs physics models as a function of a parameter.
    
    Args:
        param_name: Name of parameter to vary
        param_values: Array of parameter values to test
        base_params: Dictionary of base parameters
        
    Returns:
        chi_constant_list: List of constant chi arrays
        chi_physics_list: List of physics chi arrays
    """
    chi_constant_list = []
    chi_physics_list = []
    
    calc = ChiShiftCalculator()
    
    for param_val in param_values:
        # Update parameter
        params = base_params.copy()
        params[param_name] = param_val
        
        # Constant model (always the same)
        chi_constant = np.array([0, -0.25, -0.5, -0.75])
        chi_constant_list.append(chi_constant)
        
        # Physics model
        chi_physics = calc.calculate_chi_shifts(
            f_r=params['f_r'],
            f_q=params['f_q'], 
            anharmonicity=params['anharmonicity'],
            g=params['coupling_strength'],
            num_levels=4
        )
        chi_physics_list.append(chi_physics)
    
    return chi_constant_list, chi_physics_list

# Base parameters
base_params = {
    'f_r': 7000,
    'f_q': 5000,
    'anharmonicity': -250,
    'coupling_strength': 100
}

# Parameter ranges to test
param_tests = {
    'anharmonicity': np.linspace(-400, -100, 10),
    'coupling_strength': np.linspace(50, 150, 10),
    'f_q': np.linspace(4500, 5500, 10)
}

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for idx, (param_name, param_values) in enumerate(param_tests.items()):
    chi_const, chi_phys = compare_models_vs_parameter(param_name, param_values, base_params)
    
    # Plot chi differences for each level
    ax1 = axes[idx * 2]
    ax2 = axes[idx * 2 + 1]
    
    colors = ['blue', 'red', 'green', 'purple']
    
    # Chi values vs parameter
    for level in range(4):
        chi_const_level = [chi[level] for chi in chi_const]
        chi_phys_level = [chi[level] for chi in chi_phys]
        
        ax1.plot(param_values, chi_const_level, '--', color=colors[level], 
                alpha=0.7, label=f'Constant χ_{level}')
        ax1.plot(param_values, chi_phys_level, '-', color=colors[level], 
                linewidth=2, label=f'Physics χ_{level}')
    
    ax1.set_xlabel(param_name.replace('_', ' ').title())
    ax1.set_ylabel('Chi Shift (MHz)')
    ax1.set_title(f'Chi Shifts vs {param_name.replace("_", " ").title()}')
    ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax1.grid(True, alpha=0.3)
    
    # Relative differences
    for level in range(1, 4):  # Skip level 0 (always 0)
        chi_const_level = np.array([chi[level] for chi in chi_const])
        chi_phys_level = np.array([chi[level] for chi in chi_phys])
        
        rel_diff = (chi_phys_level - chi_const_level) / np.abs(chi_const_level) * 100
        ax2.plot(param_values, rel_diff, 'o-', color=colors[level], 
                linewidth=2, label=f'Level {level}')
    
    ax2.set_xlabel(param_name.replace('_', ' ').title())
    ax2.set_ylabel('Relative Difference (%)')
    ax2.set_title(f'Physics vs Constant Model Difference')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.axhline(0, color='black', linestyle='-', alpha=0.3)

plt.tight_layout()
plt.show()

print("Parameter Sensitivity Summary:")
print("\n• Anharmonicity: Strongly affects higher-level chi shifts")
print("• Coupling Strength: Scales all chi shifts proportionally")
print("• Qubit Frequency: Changes detuning, affecting all levels")
print("\nThe physics model automatically adapts to parameter changes,")
print("while the constant model remains fixed regardless of actual system parameters.")

## 5. Migration Guide

For users wanting to migrate from constant to physics-based chi models, here's a practical guide.

In [ ]:
print("Migration Guide: Constant to Physics-Based Chi Models")
print("=" * 55)

print("\n1. LEGACY APPROACH (Constant Chi):")
print("""
simulator = DispersiveReadoutSimulatorSyntheticData(
    f_r=7000,
    kappa=1.0,
    chis=[0, -0.25, -0.5, -0.75],  # Manual chi specification
    use_physics_model=False
)
""")

print("\n2. NEW APPROACH (Physics-Based Chi):")
print("""
# Option A: Factory method (recommended)
simulator = DispersiveReadoutSimulatorSyntheticData.from_physics_model(
    f_r=7000,                  # Resonator frequency
    f_q=5000,                  # Qubit frequency
    anharmonicity=-250,        # Transmon anharmonicity
    coupling_strength=100,     # Qubit-resonator coupling
    kappa=1.0,                # Resonator linewidth
    num_levels=4               # Number of levels to include
)

# Option B: Direct initialization
simulator = DispersiveReadoutSimulatorSyntheticData(
    f_r=7000,
    kappa=1.0,
    f_q=5000,
    anharmonicity=-250,
    coupling_strength=100,
    use_physics_model=True,    # Enable physics calculation
    chis=None                  # Let physics model calculate
)
""")

print("\n3. BACKWARDS COMPATIBILITY:")
print("""
# Physics model can be disabled to maintain legacy behavior
simulator = DispersiveReadoutSimulatorSyntheticData(
    f_r=7000,
    kappa=1.0,
    chis=[0, -0.25, -0.5, -0.75],  # Explicit chi values
    use_physics_model=False         # Disable physics model
)
""")

print("\n4. PARAMETER MAPPING:")
print("""
Required for Physics Model:
• f_r: Resonator frequency (MHz)
• f_q: Qubit frequency (MHz) 
• anharmonicity: Transmon anharmonicity (MHz, negative)
• coupling_strength: g coupling (MHz)
• kappa: Resonator linewidth (MHz)

Optional:
• num_levels: Number of transmon levels (default: 4)
• All existing pulse parameters (width, amp, etc.)
""")

print("\n5. VALIDATION CHECKLIST:")
print("""
Before migration, verify:
□ Dispersive regime: g << |f_r - f_q| (typically g/|Δ| < 0.1)
□ Reasonable anharmonicity: -400 < α < -100 MHz for transmons
□ Realistic coupling: 50 < g < 200 MHz for typical systems
□ Proper frequency ranges: 4-8 GHz for f_q, f_r
""")

# Demonstrate validation
print("\n6. EXAMPLE VALIDATION:")
calc = ChiShiftCalculator()
test_params = {
    'f_r': 7000,
    'f_q': 5000, 
    'anharmonicity': -250,
    'coupling_strength': 100
}

is_dispersive = calc.validate_dispersive_regime(
    test_params['f_r'], test_params['f_q'], test_params['coupling_strength']
)

print(f"\nValidation Results for Example Parameters:")
print(f"• Detuning: {test_params['f_r'] - test_params['f_q']} MHz")
print(f"• g/Δ ratio: {test_params['coupling_strength'] / abs(test_params['f_r'] - test_params['f_q']):.3f}")
print(f"• Dispersive regime: {'✓ Valid' if is_dispersive else '✗ Invalid'}")
print(f"• Anharmonicity: {test_params['anharmonicity']} MHz {'✓ Typical' if -400 < test_params['anharmonicity'] < -100 else '⚠ Unusual'}")

if is_dispersive:
    chi_calculated = calc.calculate_chi_shifts(**test_params)
    print(f"\nCalculated chi shifts: {chi_calculated}")
    print("✓ Ready for physics-based simulation!")
else:
    print("\n⚠ Consider adjusting parameters for better dispersive regime compliance.")

## Conclusions

This comparison demonstrates several key advantages of the physics-based chi model:

### **Accuracy Improvements**
- **Parameter-Dependent**: Chi shifts automatically adapt to actual system parameters
- **Multi-Level Physics**: Includes proper anharmonicity and coupling matrix elements
- **Realistic Scaling**: Chi differences follow $\sqrt{n}$ enhancement rather than linear steps

### **Simulation Benefits**
- **Better Fidelity Predictions**: More accurate readout fidelity estimates
- **State Discrimination**: Improved modeling of state separation
- **Protocol Development**: Better foundation for calibration procedures

### **Practical Considerations**
- **Backwards Compatible**: Existing code continues to work unchanged
- **Easy Migration**: Simple parameter mapping from experimental values
- **Validation Built-in**: Automatic dispersive regime checking

### **When to Use Each Model**

**Use Physics Model When:**
- Developing new readout protocols
- Optimizing state discrimination
- Predicting experimental performance
- Working with multi-level systems

**Use Constant Model When:**
- Maintaining legacy code
- Quick prototyping without full parameters
- Educational demonstrations
- Backwards compatibility required

The physics-based model provides a significant step forward in simulation accuracy while maintaining full compatibility with existing workflows.